# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
#from api_keys import geoapify_key
geoapify_key = "11d6d7d5e2674530b8de145d6bd06730"

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,15.96,53,31,0.45,AU,1728702351
1,1,uturoa,-16.7333,-151.4333,25.56,74,4,4.15,PF,1728702352
2,2,waitangi,-43.9535,-176.5597,10.46,97,100,4.47,NZ,1728702354
3,3,saint-pierre,-21.3393,55.4781,20.82,56,40,3.60,RE,1728702355
4,4,albany,42.6001,-73.9662,13.08,75,22,0.45,US,1728702341


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
        "Lng",
        "Lat",
        geo = True,
        tiles="OSM"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
#city_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= 22) & (city_data_df["Max Temp"] < 32)
#                               & (city_data_df["Country"] == "US") & (city_data_df["Wind Speed"] < 5)]
# Narrow down cities that fit criteria and drop any results with null values
hotel_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
hotel_df = hotel_df.dropna()
# Display sample data
hotel_df




# Drop any rows with null values
#city_data_df.dropna()

# Display sample data
#city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,wailua homesteads,22.0669,-159.3780,26.44,84,0,2.57,US,1728702414
67,67,ajdabiya,30.7554,20.2263,26.70,45,0,3.18,LY,1728702426
90,90,san patricio,28.0170,-97.5169,21.97,67,0,2.81,US,1728702452
92,92,lihue,21.9789,-159.3672,26.77,78,0,2.57,US,1728702454
198,198,siwa oasis,29.2041,25.5195,26.23,36,0,2.30,EG,1728702573
201,201,al abyar,32.1900,20.5965,23.93,46,0,3.30,LY,1728702554
222,222,laguna,38.4210,-121.4238,21.80,50,0,3.60,US,1728702602
255,255,rosetta,31.4044,30.4164,22.16,83,0,1.67,EG,1728702642
361,361,sangin,32.0728,64.8359,21.88,16,0,2.10,AF,1728702767
366,366,khuzdar,27.7384,66.6434,22.39,24,0,0.91,PK,1728702773


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Code used to drop columns from df 
hotel_df = hotel_df.drop(columns=["City_ID", "Cloudiness", "Max Temp", "Date", "Wind Speed"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
57,wailua homesteads,22.0669,-159.3780,84,US,
67,ajdabiya,30.7554,20.2263,45,LY,
90,san patricio,28.0170,-97.5169,67,US,
92,lihue,21.9789,-159.3672,78,US,
198,siwa oasis,29.2041,25.5195,36,EG,
201,al abyar,32.1900,20.5965,46,LY,
222,laguna,38.4210,-121.4238,50,US,
255,rosetta,31.4044,30.4164,83,EG,
361,sangin,32.0728,64.8359,16,AF,
366,khuzdar,27.7384,66.6434,24,PK,


In [50]:
# Reposition Columns to match example
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]]

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
57,wailua homesteads,US,22.0669,-159.3780,84,
67,ajdabiya,LY,30.7554,20.2263,45,
90,san patricio,US,28.0170,-97.5169,67,
92,lihue,US,21.9789,-159.3672,78,
198,siwa oasis,EG,29.2041,25.5195,36,
201,al abyar,LY,32.1900,20.5965,46,
222,laguna,US,38.4210,-121.4238,50,
255,rosetta,EG,31.4044,30.4164,83,
361,sangin,AF,32.0728,64.8359,16,
366,khuzdar,PK,27.7384,66.6434,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000

params = {"catagories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: No hotel found
ajdabiya - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
siwa oasis - nearest hotel: No hotel found
al abyar - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
rosetta - nearest hotel: No hotel found
sangin - nearest hotel: No hotel found
khuzdar - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
tabas - nearest hotel: No hotel found
tukrah - nearest hotel: No hotel found
mansa - nearest hotel: No hotel found
anar darah - nearest hotel: No hotel found
sidi khaled - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
57,wailua homesteads,US,22.0669,-159.3780,84,No hotel found
67,ajdabiya,LY,30.7554,20.2263,45,No hotel found
90,san patricio,US,28.0170,-97.5169,67,No hotel found
92,lihue,US,21.9789,-159.3672,78,No hotel found
198,siwa oasis,EG,29.2041,25.5195,36,No hotel found
201,al abyar,LY,32.1900,20.5965,46,No hotel found
222,laguna,US,38.4210,-121.4238,50,No hotel found
255,rosetta,EG,31.4044,30.4164,83,No hotel found
361,sangin,AF,32.0728,64.8359,16,No hotel found
366,khuzdar,PK,27.7384,66.6434,24,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE